In [70]:
import re

import requests

from lxml import html
import pandas as pd

from slugify import slugify

In [75]:
AMOUNT_RE = re.compile('^([-\d\.]+)')

In [79]:
cookies = {
    'ope_csrf_': '2d17ee8adc7dfbb0e40fec0ecd0e6617',
    'session': '9dd2ccac006f719b658e6f0e69ace726f6eec1b8',
}

headers = {
    'Origin': 'https://transpay.opekepe.gr',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-US,en;q=0.8,de;q=0.6',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.94 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://transpay.opekepe.gr/',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive',
    'DNT': '1',
}

LIMIT = 5000
YEAR = 2014

data = {'upper': ['-1'], 'lower': ['-1'], 'total_rows': ['1074916'],
        'csrf_token_': ['2d17ee8adc7dfbb0e40fec0ecd0e6617'], 'submit': ['1'],
        'limit': [str(LIMIT)], 'year': [str(YEAR), str(YEAR)]}

In [81]:
def get_rows_starting(start_row):
    response = requests.post('https://transpay.opekepe.gr/welcome/search/%s' % start_row,
                             headers=headers, cookies=cookies, data=data)
    root = html.fromstring(response.content.decode('unicode-escape'))
    for tr in root.xpath('.//table/tbody/tr'):
        tds = [td.text_content() for td in tr.xpath('./td')]
        if 'No results' in tds[1]:
            break
        yield {
            'recipient_id': 'GR-%s-%s' % (YEAR, tds[0]),
            'recipient_name': tds[1] if not tds[1].startswith('ANONYMOUS') else '',
            'recipient_location': tds[2],
            'scheme': '%s (%s)' % (tds[3], tds[4]),
            'amount': float(AMOUNT_RE.search(tds[5]).group(1))
        }

In [62]:
def get_all_rows():
    start_row = 1
    while True:
        print('Starting row %d' % start_row)
        row_count = 0
        for row in get_rows_starting(start_row):
            row_count += 1
            yield row
        if row_count == 0:
            break
        start_row += row_count

In [82]:
df = pd.DataFrame(get_all_rows())
df.head()

Starting row 1
> <ipython-input-81-fcd4b6322d95>(5)get_rows_starting()
-> root = html.fromstring(response.content.decode('unicode-escape'))
(Pdb) response.text
u''
(Pdb) c


XMLSyntaxError: line 1093: Invalid char in CDATA 0x0